In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
PATH = './results/final/data_train_merged_23_10.csv'

In [3]:
def bulkInsert(records):
    try:
        connection = psycopg2.connect(user="postgres",
                                    password="migue219",
                                    host="127.0.0.1",
                                    port="5432",
                                    database="landslide_project")
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO public.landslide_inventory(
	                            id, version, event_date, location, satellite, landslide, rock_geo, rock_period, fis_nomr, rock_type, 
                                soil_rec_symbol, soil_rec_mat_origin, soil_rec_sew_class, soil_rec_tex_sup, soil_rec_c_tex_sup, 
                                soil_rec_tex_sub, soil_rec_c__tex_sub, soil_rec_risk_eros, soil_tax, soil_use, 
                                precipitation_mm_last_7_days, elevation_mean, slope_mean, aspect_mean, placurv_mean, procurv_mean, 
                                elevation, slope, aspect, placurv, procurv)
	                            VALUES (%s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                                %s, %s, %s, %s, %s, %s, %s, %s); """

        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into table")

    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into table {}".format(error))

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [4]:
data = pd.read_csv(PATH, index_col=0)
data = data.where(data.notnull(), None)

In [5]:
records_to_insert = []
for index, row in data.iterrows():
    elevation_array, slope_array, aspect_array, placurv_array, procurv_array = [], [], [], [], []
    for i in range(1, 26):
        elevation_array.append(row['elevation_{}'.format(i)])
        slope_array.append(row['slope_{}'.format(i)])
        aspect_array.append(row['aspect_{}'.format(i)])
        placurv_array.append(row['placurv_{}'.format(i)])
        procurv_array.append(row['procurv_{}'.format(i)])
    elevation_mean = np.mean(elevation_array)
    slope_mean = np.mean(slope_array)
    aspect_mean = np.mean(aspect_array)
    placurv_mean = np.mean(placurv_array)
    procurv_mean = np.mean(procurv_array)
    records_to_insert.append((row['landslide_id'], 1, row['event_date'], row['longitude'], row['latitude'], row['satellite'], row['landslide'], 
    row['rock_geo'], row['rock_period'], row['fis_nomr'], row['rock_type'], row['soil_rec_symbol'], row['soil_rec_mat_origin'], 
    row['soil_rec_sew_class'], row['soil_rec_tex_sup'], row['soil_rec_c_tex_sup'], row['soil_rec_tex_sub'], row['soil_rec_c__tex_sub'], 
    row['soil_rec_risk_eros'], row['soil_tax'], row['soil_use'], row['precipitation_mm_last_7_days'], elevation_mean, 
    slope_mean, aspect_mean, placurv_mean, procurv_mean, elevation_array, slope_array, aspect_array, 
    placurv_array, procurv_array))

In [6]:
print(len(records_to_insert))

9742


In [7]:
bulkInsert(records_to_insert)

9742 Record inserted successfully into table
PostgreSQL connection is closed
